# Base classes
> Base configuration classes and utilities for the logging library.

In [3]:
#| default_exp client.base

In [4]:
#| export
from dc_logger.client.Log import Log

ModuleNotFoundError: No module named 'dc_logger'

In [ ]:
#| exporti


from typing import Optional, List, Dict, Any, Literal
from dataclasses import dataclass, field
from abc import ABC, abstractmethod
from enum import Enum



In [ ]:
#| export

@dataclass
class LoggerSettings(ABC):
    """abstract base configuration for logging configuration settings"""

    format: str = "json"  # json, text
    batch_size: int = 100
    flush_interval: int = 30  # seconds
    correlation_enabled: bool = True
    include_traceback: bool = True
    max_buffer_size: int = 1000

## Service Handler defines 'route functions' for how to interact with a service.

In [ ]:
#| export


# Type for valid output modes
OutputMode = Literal["cloud", "console", "file", "multi"]


@dataclass
class ServiceConfig(ABC):
    """abstract base class for service-specific configuration settings"""

    output_mode: OutputMode

    def __post_init__(self):
        self.validate_config()


    @abstractmethod
    def validate_config(self) -> bool:
        """Validate the configuration"""
        raise NotImplementedError()


@dataclass
class ServiceHandler(ABC):
    """defines how a handler communicates with services to create logs"""

    service_config: Optional[ServiceConfig] = None # has authentication and connection details to service1

    @classmethod
    def from_config(cls, service_config: ServiceConfig):
        
        hc = cls(
            service_config = service_config
            
        )
        
        # if hasattr(config, 'to_platform_config') and callable(getattr(config, 'to_platform_config')):
        #     hc.platform_config = config.to_platform_config()
        return hc
    

    @abstractmethod
    async def write(self, entries: List[Log]) -> bool:
        """Write log entries to destination"""
        pass

    @abstractmethod
    async def flush(self) -> bool:
        """Flush any buffered entries"""
        pass

    async def close(self):
        """Clean up resources"""
        pass


In [ ]:
#| export

    
@dataclass
class LogHandler(ABC):
    """defines when, how and where to send log entries"""

    log_config: LoggerSettings
    service_handler: ServiceHandler

    log_level: LogLevel = LogLevel.INFO #minimum log level to log.

    log_method : List[str] = field(default_factory=lambda: ["POST", "PUT", "DELETE", "PATCH"]) # filtered list of API requests to log, generally won't log GET requests

    @abstractmethod
    async def write(self, entries: List[Log]) -> bool:
        """Write log entries to destination"""
        pass

    @abstractmethod
    async def flush(self) -> bool:
        """Flush any buffered entries"""
        pass

    async def close(self):
        """Clean up resources"""
        pass



In [ ]:
#| export

@dataclass 
class Logger:
    """ should receive log entries and send them to all handlers.  handlers will use log_level and log_method to determine which logs to send"""
    handlers: List[LogHandler] = field(default_factory=list)
    pretty_print: bool = False  # Pretty print JSON for development

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export('./base.ipynb')